# BNN指标

数据集 MNIST

推断方法 BBB

In [1]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))

import BayesKeras
from BayesKeras import PosteriorModel
from BayesKeras import analyzers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np

D:\Users\T470\Anaconda3\envs\CertifiableBayesianInference\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
dict = {'eps': 0.11, 'lam': 0.25, 'rob': 2, 'opt': 'BBB', 'gpu': '1'}
# robustness mode, 0:标准 1:IBP 2:PGD
rob = dict['rob']
# the variational inference method
opt = dict['opt']
inference = opt

## accuracy

### 查看模型

训练了以下BNN：

输入层 28\*28 = 784个神经元

<img src="https://gitee.com/oeong/picgo/raw/master/images/20211220171155.png" alt="image-20211219223415584" style="zoom:60%;" />

In [5]:
"""
model = Sequential()
model.add(Dense(512, activation="relu", input_shape=(None, None, 28 * 28)))
model.add(Dense(10, activation="softmax"))
"""
model = PosteriorModel("%s_FCN_Posterior_%s_mine" % (inference, rob))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, None, None, 512)   401920    
                                                                 
 dense_9 (Dense)             (None, None, None, 10)    5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 


==========================posterior_mean_start==========================
[<tf.Tensor: shape=(784, 512), dtype=float32, numpy=
 array([[-0.00117524, -0.00130718, -0.00062442, ..., -0.00214078,
         -0.00109595,  0.00662703],
        [-0.00140605, -0.00124328, -0.00113443, ..., -0.00153678,
         -0.00130616,  0.00883936],
        [-0.0017624 , -0.00128564, -0.00082267, ..., -0.00111104,
         

全连接层神经网络的`Param`，即每层权重的个数

Param = （输入数据维度+1）* 神经元个数，加1是因为每层网络的神经元都有一个偏置bias

第一个Dense层，输入维度28\*28=784，param=785\*512=401920

第二个Dense层，输入维度512, param=513\*10=5130

- 第二个Dense层的权重w、偏置b的后验**均值**分布
```
[ <tf.Tensor: shape=(512, 10), dtype=float32, numpy=
 array([[ 0.0185502 , -0.01161555, -0.04420954, ..., -0.00332918,
         -0.01537434,  0.00688318],
        [ 0.01812534,  0.02651611, -0.06108603, ...,  0.01529146,
         -0.01838634, -0.00816697],
        [-0.04614896,  0.06859293, -0.01545104, ...,  0.01977891,
         -0.00673474, -0.00097881],
        ...,
        [ 0.00217267, -0.00978215, -0.0012502 , ..., -0.01652447,
         -0.01060206,  0.00981593],
        [-0.02092285,  0.05196731, -0.01296113, ...,  0.02238781,
         -0.01631541,  0.02332588],
        [-0.11078487,  0.88213193, -0.4701395 , ..., -0.06491183,
         -0.58317137,  0.00915936]], dtype=float32)>
 <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([ 0.02872071, -0.02288143,  0.21539441, -0.1829949 ,  0.00210195,
         0.351896  , -0.01249683, -0.00474732,  0.10593052, -0.15292557],
       dtype=float32)>                                                   ]
```

- 第二个Dense层的权重w、偏置b的后验**方差**分布
```
[ <tf.Tensor: shape=(512, 10), dtype=float32, numpy=
 array([[0.2629409 , 0.26680163, 0.26598117, ..., 0.26652598, 0.26963538,
         0.26568806],
        [0.2709221 , 0.27090704, 0.2669378 , ..., 0.26847348, 0.26890156,
         0.2680423 ],
        [0.26738682, 0.2632878 , 0.2772156 , ..., 0.27254277, 0.2768247 ,
         0.26886734],
        ...,
        [0.2717316 , 0.26931196, 0.2694975 , ..., 0.26624396, 0.26710454,
         0.26795733],
        [0.26761043, 0.27089885, 0.26964507, ..., 0.2710983 , 0.2717969 ,
         0.2722351 ],
        [0.26733303, 0.21829   , 0.23816003, ..., 0.25891876, 0.2427067 ,
         0.2670901 ]], dtype=float32)>
 <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([0.7358247 , 0.6331638 , 0.66349566, 0.982099  , 0.6567518 ,
        0.64802605, 0.743553  , 0.7543861 , 0.69611764, 0.87573653],
       dtype=float32)>                                              ]
```

### 输入

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_test = X_test / 255.
X_test = X_test.astype("float32").reshape(-1, 28*28)

In [4]:
X_test[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [23]:
y_test[0]

7

In [6]:
from tqdm import trange

loss = tf.keras.losses.SparseCategoricalCrossentropy()

num_images = 500

### 输出

In [8]:
accuracy = tf.keras.metrics.Accuracy()
preds = model.predict(X_test[0:500])


==========================predict_out_start==========================
tf.Tensor(
[[2.52925972e-22 9.91964859e-15 1.11175709e-14 ... 1.00000000e+00
  3.89382284e-20 7.21768641e-21]
 [3.47048887e-12 1.35598469e-07 9.99999583e-01 ... 3.97243349e-10
  3.50059148e-10 1.12642829e-14]
 [2.55755703e-14 1.00000000e+00 2.78211444e-14 ... 1.96422544e-12
  7.22589381e-17 8.94728763e-14]
 ...
 [4.76991158e-09 7.87215868e-06 5.48555022e-07 ... 1.13199621e-06
  1.02209866e-04 1.99844317e-06]
 [9.99999583e-01 2.37285755e-12 1.08157863e-07 ... 9.44622019e-11
  1.02477131e-07 2.14748454e-07]
 [1.13907843e-08 1.57176563e-10 7.82225218e-09 ... 2.78890782e-11
  6.68151915e-06 2.80340161e-11]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================



In [24]:
preds[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([8.9725687e-18, 2.5878758e-15, 2.8447819e-16, 1.3929219e-17,
       5.6852981e-22, 7.4145332e-19, 2.1005616e-26, 1.0000000e+00,
       1.1781463e-21, 1.1392881e-17], dtype=float32)>

In [9]:
y_pred = np.argmax(preds, axis=1)  # 最大元素的索引值
y_pred

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 9, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 5, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 8,
       8, 7, 2, 3, 0, 2, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 0, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 4, 2, 9, 2, 0, 4, 0, 0, 2, 8, 1,

In [10]:
y_true = y_test[0:500]
y_true

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

In [11]:
accuracy.update_state(y_pred, y_true)
print("%s Accuracy: " % (inference), accuracy.result())

BBB Accuracy:  tf.Tensor(0.964, shape=(), dtype=float32)


## robustness

鲁棒性：在异常和危险情况下系统生存的能力

对输入数据加入扰动后的准确率

In [27]:
accuracy = tf.keras.metrics.Accuracy()
# 用PGD去近似输入点最坏情况的扰动
adv = analyzers.PGD(model, X_test[0:500], eps=0.1, loss_fn=loss, num_models=10)


==========================predict_out_start==========================
tf.Tensor(
[[7.7724629e-21 4.8051036e-15 4.3843955e-16 ... 1.0000000e+00
  1.7190970e-20 3.2167871e-20]
 [2.8221557e-12 1.6829057e-08 9.9999869e-01 ... 3.1424516e-09
  3.6584971e-10 5.2017361e-15]
 [3.0815835e-14 1.0000000e+00 2.5472999e-16 ... 1.6540546e-14
  3.6858824e-17 8.3566238e-15]
 ...
 [6.1186364e-07 3.2452928e-05 2.7019371e-09 ... 4.2156274e-09
  1.0880101e-05 3.5918958e-08]
 [9.9999976e-01 1.9530251e-11 3.3070169e-08 ... 2.3915035e-11
  2.1236397e-07 1.0866300e-08]
 [1.1610198e-08 3.4690136e-11 3.5941290e-09 ... 7.0366317e-11
  7.9573175e-07 2.2073676e-11]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================


==========================predict_out_start==========================
tf.Tensor(
[[1.2240667e-21 1.9158873e-15 6.8649263e-15 ... 1.0000000e+00
  1.4595172e-20 2.9654406e-20]
 [1.2598285e-10 1.4081834e-08 9.9999976e-01 ... 2.7284824e-10
  5.4771618e-09


==========================predict_out_start==========================
tf.Tensor(
[[8.69149426e-22 4.63233544e-14 4.60190353e-16 ... 1.00000000e+00
  7.17870633e-20 8.87595817e-18]
 [6.03454717e-11 1.46525383e-06 9.98977244e-01 ... 3.81690552e-06
  4.23987831e-06 4.46306880e-12]
 [8.93676951e-14 1.00000000e+00 1.94382593e-14 ... 1.43099540e-12
  1.20109444e-16 5.37501901e-13]
 ...
 [3.31915260e-07 1.99418773e-05 4.19272013e-08 ... 5.59037368e-08
  1.68917293e-04 2.91992080e-07]
 [9.99998450e-01 9.80044518e-11 4.82728133e-08 ... 2.81767630e-11
  1.39867416e-06 1.07773104e-07]
 [5.55864652e-08 1.06704396e-08 1.22767583e-08 ... 6.61982080e-10
  4.57242248e-04 2.38780280e-11]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================


==========================predict_out_start==========================
tf.Tensor(
[[2.7111849e-20 2.5238936e-14 4.9241893e-13 ... 1.0000000e+00
  4.7215072e-20 9.5169177e-19]
 [1.6746260e-07 1.9888774e-05 9.9995083e-


==========================predict_out_start==========================
tf.Tensor(
[[1.3192299e-19 3.4212883e-13 3.1663139e-14 ... 1.0000000e+00
  2.4306679e-19 5.2437170e-20]
 [1.2672947e-09 3.9951243e-05 9.9472439e-01 ... 1.4747762e-08
  9.3938411e-08 8.9971390e-12]
 [6.1857708e-13 1.0000000e+00 2.5688102e-14 ... 1.6704252e-09
  1.8382718e-13 1.1433720e-11]
 ...
 [3.4133202e-06 1.0346374e-03 8.2300518e-05 ... 1.8574343e-05
  1.6644743e-01 1.2719044e-03]
 [9.8816472e-01 1.6249510e-07 4.8006375e-05 ... 6.3281242e-09
  1.1782961e-02 1.4264405e-06]
 [8.9137675e-06 1.3447122e-04 1.4116224e-06 ... 4.1156905e-08
  2.6045218e-02 2.4150179e-07]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================


==========================predict_out_start==========================
tf.Tensor(
[[3.31454159e-21 2.03444185e-15 3.01754471e-14 ... 1.00000000e+00
  4.37648286e-19 7.78271103e-20]
 [2.77521811e-10 2.85649767e-05 9.73688066e-01 ... 3.37584424e-07
  1.1


==========================predict_out_start==========================
tf.Tensor(
[[1.1465200e-19 2.9748900e-13 9.3837342e-15 ... 1.0000000e+00
  7.9716537e-18 4.6217906e-18]
 [2.1845560e-06 7.2157432e-05 9.6373856e-01 ... 5.0448551e-07
  4.2080965e-06 3.1614190e-11]
 [1.3309803e-13 1.0000000e+00 5.0273947e-15 ... 8.0537878e-14
  7.9045301e-15 2.4943348e-12]
 ...
 [2.1919261e-05 1.0472482e-02 3.7779122e-05 ... 2.1084439e-05
  5.3104979e-01 1.0451715e-03]
 [9.5177591e-01 3.3279548e-07 7.0799535e-05 ... 4.1553022e-08
  4.7959503e-02 3.3914721e-06]
 [2.6863697e-04 6.9528560e-06 3.0239293e-05 ... 9.5312538e-07
  6.2653267e-01 2.9865190e-05]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================


==========================predict_out_start==========================
tf.Tensor(
[[1.60447416e-20 3.65119350e-12 1.28987776e-15 ... 1.00000000e+00
  5.60623720e-21 1.58691480e-21]
 [4.98916686e-10 2.08875026e-05 9.88332272e-01 ... 1.89686816e-07
  1.4


==========================predict_out_start==========================
tf.Tensor(
[[1.36792667e-19 1.11747795e-14 7.25026406e-15 ... 1.00000000e+00
  4.53648547e-21 4.59071798e-19]
 [3.52090615e-06 3.77015397e-03 7.12701261e-01 ... 1.56232181e-05
  2.31293001e-04 2.24722655e-07]
 [1.87566403e-11 1.00000000e+00 7.34592170e-14 ... 6.37979669e-11
  1.45233606e-16 5.75178468e-14]
 ...
 [1.77471757e-05 6.12713164e-04 2.68533011e-04 ... 7.60288503e-06
  9.65065241e-01 1.34030869e-03]
 [7.72149980e-01 4.36396385e-06 5.72086021e-04 ... 1.09484297e-07
  2.26906717e-01 2.24361778e-04]
 [4.02657752e-05 5.73719672e-06 1.31228415e-04 ... 2.06083470e-07
  9.65003967e-01 5.01865870e-05]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================


==========================predict_out_start==========================
tf.Tensor(
[[4.62864339e-18 9.21542031e-15 1.25851105e-14 ... 1.00000000e+00
  8.05716310e-20 5.50724806e-19]
 [3.50157201e-08 4.49439394e-04 9.0


==========================predict_out_start==========================
tf.Tensor(
[[4.10774735e-23 1.05644404e-16 2.57018214e-15 ... 1.00000000e+00
  4.99066004e-23 2.96949310e-19]
 [8.95055763e-08 5.74611500e-03 7.02926099e-01 ... 3.51407711e-04
  8.09890407e-05 7.97235629e-08]
 [2.80848209e-11 1.00000000e+00 1.00883462e-14 ... 1.18241962e-14
  2.64888209e-15 2.25297296e-14]
 ...
 [6.34923481e-06 4.70360355e-05 9.07310750e-05 ... 1.39575968e-05
  9.96664524e-01 1.56762544e-03]
 [3.49362731e-01 9.15714111e-07 1.40575115e-02 ... 6.58066384e-08
  6.36021256e-01 4.97198234e-05]
 [3.54242479e-06 7.31444597e-05 2.85464539e-05 ... 1.39317905e-07
  9.88559306e-01 3.36914891e-05]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================


==========================predict_out_start==========================
tf.Tensor(
[[5.3390439e-22 7.2607765e-16 3.0079247e-15 ... 1.0000000e+00
  1.3214598e-19 2.7656926e-20]
 [5.7797187e-08 9.7232981e-04 6.0086364e-

In [26]:
adv[0]

array([0.01      , 0.01      , 0.01      , 0.        , 0.        ,
       0.01      , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01      , 0.        , 0.        ,
       0.01      , 0.01      , 0.01      , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.01      ,
       0.        , 0.01      , 0.01      , 0.01      , 0.        ,
       0.01      , 0.        , 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.        , 0.        ,
       0.01      , 0.01      , 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.01      , 0.        ,
       0.01      , 0.        , 0.01      , 0.        , 0.01      ,
       0.01      , 0.        , 0.        , 0.        , 0.        ,
       0.01      , 0.01      , 0.        , 0.        , 0.        ,
       0.01      , 0.01      , 0.01      , 0.        , 0.        ,
       0.        , 0.01      , 0.01      , 0.        , 0.     

In [15]:
preds = model.predict(adv)


==========================predict_out_start==========================
tf.Tensor(
[[8.9725687e-18 2.5878758e-15 2.8447819e-16 ... 1.0000000e+00
  1.1781463e-21 1.1392881e-17]
 [8.6201635e-07 3.2586415e-04 8.7566340e-01 ... 6.4967262e-06
  9.7265947e-05 9.8467394e-09]
 [6.7156774e-15 1.0000000e+00 1.4527890e-13 ... 5.4296064e-13
  8.3715502e-17 2.2251134e-14]
 ...
 [2.2360329e-05 1.1497434e-03 5.1026291e-05 ... 1.0083754e-05
  9.1637999e-01 6.4612564e-04]
 [7.9368949e-01 1.7854265e-05 1.7989904e-03 ... 2.3340272e-06
  2.0321801e-01 1.1142129e-03]
 [8.6181200e-05 1.1885283e-04 2.0881642e-03 ... 2.6431605e-06
  9.3044400e-01 5.3449538e-05]], shape=(500, 10), dtype=float32)
==========================predict_out_end==========================



In [16]:
y_pred = np.argmax(preds, axis=1)
y_pred

array([7, 2, 1, 0, 4, 1, 4, 9, 2, 7, 0, 8, 9, 0, 1, 5, 9, 7, 3, 4, 7, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 3, 1, 1, 2, 4, 1,
       5, 8, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 4, 5, 7, 2, 5, 2, 9, 9,
       6, 4, 3, 0, 7, 0, 0, 8, 1, 4, 3, 7, 8, 7, 9, 6, 2, 7, 8, 4, 7, 5,
       6, 1, 3, 6, 4, 3, 1, 4, 9, 3, 6, 9, 6, 0, 5, 4, 5, 9, 2, 1, 9, 4,
       8, 1, 3, 9, 2, 9, 4, 4, 9, 8, 5, 6, 7, 6, 4, 4, 0, 5, 8, 5, 6, 6,
       5, 2, 8, 1, 0, 1, 6, 4, 6, 7, 2, 1, 9, 1, 8, 2, 0, 2, 4, 8, 5, 5,
       1, 5, 6, 0, 2, 1, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 3, 3, 2, 1,
       1, 8, 1, 8, 1, 8, 8, 0, 8, 4, 2, 5, 0, 1, 8, 8, 0, 4, 0, 2, 1, 6,
       4, 2, 8, 6, 1, 1, 1, 3, 5, 5, 2, 9, 4, 8, 9, 9, 9, 0, 3, 6, 5, 5,
       7, 2, 8, 7, 1, 2, 8, 4, 1, 7, 5, 8, 5, 7, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 4, 2, 6, 0, 6, 4, 2, 4, 8, 9, 5, 7, 7, 2, 8, 2, 0, 8, 1, 7, 7,
       8, 1, 0, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 8, 5, 9, 2,
       6, 4, 1, 8, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 6,

In [17]:
y_true = y_test[0:500]
y_true

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

In [19]:
accuracy.update_state(y_pred, y_true)
fgsm = accuracy.result()
print("FGSM Robustness: ", accuracy.result())

FGSM Robustness:  tf.Tensor(0.754, shape=(), dtype=float32)
